In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(temperature=0.1)


memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


examples = [
    {"question": "탑건", "answer": """🛩️👨‍✈️🔥"""},
    {"question": "대부", "answer": """👨‍👨‍👦🔫🍝"""},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What is the plot of {question}?"),
        ("ai", "{answer}"),
    ]
)
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI that converts movie titles to 3 emojis.",
        ),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        (
            "human",
            (
                "Given a {question}, respond with exactly 3 emojis that"
                " represent it."
            ),
        ),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)


invoke_chain("탑건")
invoke_chain("대부")


def recall_chain():
    history = memory.load_memory_variables({})["history"]
    print("History:", history)

🛩️👨‍✈️🔥
👨‍👨‍👦🔫🍝


In [6]:
invoke_chain("아바타")

🌿🪐🔵


In [7]:
invoke_chain("라이언킹")

🦁👑🌅


In [8]:
recall_chain()

History: [HumanMessage(content='탑건'), AIMessage(content='🛩️👨\u200d✈️🔥'), HumanMessage(content='대부'), AIMessage(content='👨\u200d👨\u200d👦🔫🍝'), HumanMessage(content='아바타'), AIMessage(content='🌿🪐🔵'), HumanMessage(content='라이언킹'), AIMessage(content='🦁👑🌅')]
